In [1]:
# Imports and Setup
import os
import glob
import json
from PIL import Image
from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from diffusers import StableDiffusionPipeline, DDPMScheduler
from diffusers.optimization import get_scheduler
from accelerate import Accelerator
from accelerate.utils import set_seed

from peft import LoraConfig, get_peft_model
from transformers import CLIPTokenizer

#  multiprocessing start method
import multiprocessing as mp
mp.set_start_method("spawn", force=True)


2025-05-28 20:37:36.525620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748464656.702673      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748464656.756501      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
IMAGE_FOLDER = "/kaggle/input/ffhq-1024x1024/images1024x1024"
TEXT_FOLDER = "/kaggle/input/ffhq-20-to-30-descriptionss"
OUTPUT_DIR = "epicphotogasm_lora"
RESOLUTION = 512
BATCH_SIZE = 4
GRAD_ACCUM_STEPS = 4
NUM_EPOCHS = 5
LEARNING_RATE = 5e-5
LR_SCHEDULER = "cosine"
MIXED_PRECISION = "fp16"
SEED = 42
MAX_SAMPLES = 10000
SAVE_EVERY_N_STEPS = 1000

set_seed(SEED)



In [3]:
# Dataset
class CaptionedImageDataset(Dataset):
    def __init__(self, text_folder, image_folder, size=1024, max_samples=None):
        self.pairs = []
        for txt_file in sorted(glob.glob(os.path.join(text_folder, "*.txt"))):
            base_name = os.path.splitext(os.path.basename(txt_file))[0]
            for ext in ['.png', '.jpg', '.jpeg']:
                img_path = os.path.join(image_folder, base_name + ext)
                if os.path.exists(img_path):
                    self.pairs.append((img_path, txt_file))
                    break

        if max_samples:
            self.pairs = self.pairs[:max_samples]

        self.transform = transforms.Compose([
            transforms.Resize(size),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, txt_path = self.pairs[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        with open(txt_path, 'r', encoding='utf-8') as f:
            prompt = f.read().strip()
        return {"image": image, "prompt": prompt}


In [4]:
# Load Model
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
)
pipe.vae.requires_grad_(False)
pipe.text_encoder.requires_grad_(False)
pipe.text_encoder = pipe.text_encoder.to("cuda")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
# Apply LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "to_q", "to_k", "to_v", "to_out.0",
        "proj_in", "proj_out",
        "conv1", "conv2", "conv_shortcut"
    ],
    lora_dropout=0.1,
    bias="none",
)
pipe.unet = get_peft_model(pipe.unet, lora_config)
pipe.unet.print_trainable_parameters()
pipe.unet.enable_gradient_checkpointing()


trainable params: 12,374,016 || all params: 871,894,980 || trainable%: 1.4192


In [6]:
# Dataset & DataLoader
dataset = CaptionedImageDataset(TEXT_FOLDER, IMAGE_FOLDER, size=RESOLUTION, max_samples=MAX_SAMPLES)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)

# Optimizer & Scheduler
optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), weight_decay=1e-4)
lr_scheduler = get_scheduler(LR_SCHEDULER, optimizer=optimizer, num_warmup_steps=100, num_training_steps=NUM_EPOCHS * (len(dataloader) // GRAD_ACCUM_STEPS))
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")

accelerator = Accelerator(gradient_accumulation_steps=GRAD_ACCUM_STEPS, mixed_precision=MIXED_PRECISION, log_with="tensorboard", project_dir=OUTPUT_DIR)
pipe.unet, optimizer, dataloader, lr_scheduler = accelerator.prepare(pipe.unet, optimizer, dataloader, lr_scheduler)
tokenizer = pipe.tokenizer


In [7]:
"""
# Training Loop
def train_loop():
    global_step = 0
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    pipe.unet.train()
    vae = pipe.vae.to(accelerator.device, dtype=torch.float16)
    loss_history = []

    def clean_old_files(pattern, keep=2):
        files = sorted(glob.glob(os.path.join(OUTPUT_DIR, pattern)))
        if len(files) > keep:
            for f in files[:-keep]:
                os.remove(f)

    # Use one reference prompt from training set for generation
    reference_prompt = "A young man with short hair and a soft expression, looking directly at the camera. He is not wearing glasses."

    for epoch in range(NUM_EPOCHS):
        epoch_losses = []
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)

        for batch in progress_bar:
            with accelerator.accumulate(pipe.unet):
                images = batch["image"].to(accelerator.device, dtype=torch.float16)

                with torch.no_grad():
                    latents = vae.encode(images).latent_dist.sample() * 0.18215

                noise = torch.randn_like(latents)
                timesteps = torch.randint(
                    0, noise_scheduler.config.num_train_timesteps,
                    (latents.size(0),), device=latents.device
                )
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                encoded = tokenizer(
                    batch["prompt"],
                    padding="max_length",
                    truncation=True,
                    max_length=tokenizer.model_max_length,
                    return_tensors="pt"
                )
                input_ids = encoded.input_ids.to(accelerator.device)
                attention_mask = encoded.attention_mask.to(accelerator.device)

                encoder_hidden_states = pipe.text_encoder(input_ids=input_ids, attention_mask=attention_mask)[0]

                with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                    noise_pred = pipe.unet(
                        noisy_latents,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states
                    )[0]
                    loss = torch.nn.functional.mse_loss(noise_pred, noise)

                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(pipe.unet.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad(set_to_none=True)

                epoch_losses.append(loss.item())
                progress_bar.set_postfix(loss=loss.item(), lr=lr_scheduler.get_last_lr()[0])
                accelerator.log({"loss": loss.item(), "lr": lr_scheduler.get_last_lr()[0]}, step=global_step)

                global_step += 1

        avg_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Epoch {epoch} - Average Loss: {avg_loss:.4f}")

        if accelerator.is_main_process:
            loss_history.append({"epoch": epoch, "average_loss": avg_loss})
            with open(os.path.join(OUTPUT_DIR, "loss_log.json"), "w") as f:
                json.dump(loss_history, f, indent=2)

            lora_path = os.path.join(OUTPUT_DIR, f"lora_weights_epoch{epoch}.pth")
            torch.save({k: v for k, v in pipe.unet.state_dict().items() if "lora_" in k or "lora." in k}, lora_path)
            clean_old_files("lora_weights_epoch*.pth")

            # Generate preview image using reference prompt
            with torch.autocast("cuda"):
                generated_image = pipe(reference_prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
                gen_path = os.path.join(OUTPUT_DIR, f"preview_epoch{epoch}.png")
                generated_image.save(gen_path)

    if accelerator.is_main_process:
        torch.save({k: v for k, v in pipe.unet.state_dict().items() if "lora_" in k or "lora." in k}, os.path.join(OUTPUT_DIR, "lora_weights_last.pth"))
        print("\n Training complete. Final models saved.")


""" 

'\n# Training Loop\ndef train_loop():\n    global_step = 0\n    os.makedirs(OUTPUT_DIR, exist_ok=True)\n    pipe.unet.train()\n    vae = pipe.vae.to(accelerator.device, dtype=torch.float16)\n    loss_history = []\n\n    def clean_old_files(pattern, keep=2):\n        files = sorted(glob.glob(os.path.join(OUTPUT_DIR, pattern)))\n        if len(files) > keep:\n            for f in files[:-keep]:\n                os.remove(f)\n\n    # Use one reference prompt from training set for generation\n    reference_prompt = "A young man with short hair and a soft expression, looking directly at the camera. He is not wearing glasses."\n\n    for epoch in range(NUM_EPOCHS):\n        epoch_losses = []\n        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)\n\n        for batch in progress_bar:\n            with accelerator.accumulate(pipe.unet):\n                images = batch["image"].to(accelerator.device, dtype=torch.float16)\n\n               

In [8]:
""" 
# Start Training
train_loop()
""" 

' \n# Start Training\ntrain_loop()\n'

In [9]:
"""NUM_EPOCHS = 10

# 
loaded_lora_weights = torch.load("/kaggle/input/lora-weights-last/lora_weights_last (1).pth", map_location="cpu")
pipe.unet.load_state_dict(loaded_lora_weights, strict=False)

#  start_epoch
START_EPOCH = 5

# train_loop 
def train_loop_resume_from_5():
    global_step = 0
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    pipe.unet.train()
    vae = pipe.vae.to(accelerator.device, dtype=torch.float16)
    loss_history = []

    def clean_old_files(pattern, keep=2):
        files = sorted(glob.glob(os.path.join(OUTPUT_DIR, pattern)))
        if len(files) > keep:
            for f in files[:-keep]:
                os.remove(f)

    reference_prompt = "A young man with short hair and a soft expression, looking directly at the camera. He is not wearing glasses."

    for epoch in range(START_EPOCH, NUM_EPOCHS):
        epoch_losses = []
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)

        for batch in progress_bar:
            with accelerator.accumulate(pipe.unet):
                images = batch["image"].to(accelerator.device, dtype=torch.float16)

                with torch.no_grad():
                    latents = vae.encode(images).latent_dist.sample() * 0.18215

                noise = torch.randn_like(latents)
                timesteps = torch.randint(
                    0, noise_scheduler.config.num_train_timesteps,
                    (latents.size(0),), device=latents.device
                )
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                encoded = tokenizer(
                    batch["prompt"],
                    padding="max_length",
                    truncation=True,
                    max_length=tokenizer.model_max_length,
                    return_tensors="pt"
                )
                input_ids = encoded.input_ids.to(accelerator.device)
                attention_mask = encoded.attention_mask.to(accelerator.device)

                encoder_hidden_states = pipe.text_encoder(input_ids=input_ids, attention_mask=attention_mask)[0]

                with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                    noise_pred = pipe.unet(
                        noisy_latents,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states
                    )[0]
                    loss = torch.nn.functional.mse_loss(noise_pred, noise)

                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(pipe.unet.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad(set_to_none=True)

                epoch_losses.append(loss.item())
                progress_bar.set_postfix(loss=loss.item(), lr=lr_scheduler.get_last_lr()[0])
                accelerator.log({"loss": loss.item(), "lr": lr_scheduler.get_last_lr()[0]}, step=global_step)

                global_step += 1

        avg_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Epoch {epoch} - Average Loss: {avg_loss:.4f}")

        if accelerator.is_main_process:
            loss_history.append({"epoch": epoch, "average_loss": avg_loss})
            with open(os.path.join(OUTPUT_DIR, "loss_log.json"), "w") as f:
                json.dump(loss_history, f, indent=2)

            lora_path = os.path.join(OUTPUT_DIR, f"lora_weights_epoch{epoch}.pth")
            torch.save({k: v for k, v in pipe.unet.state_dict().items() if "lora_" in k or "lora." in k}, lora_path)
            clean_old_files("lora_weights_epoch*.pth")

            with torch.autocast("cuda"):
                generated_image = pipe(reference_prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
                gen_path = os.path.join(OUTPUT_DIR, f"preview_epoch{epoch}.png")
                generated_image.save(gen_path)

    if accelerator.is_main_process:
        torch.save({k: v for k, v in pipe.unet.state_dict().items() if "lora_" in k or "lora." in k}, os.path.join(OUTPUT_DIR, "lora_weights_last.pth"))
        print("\n Training complete. Final models saved.")

 """

'NUM_EPOCHS = 10\n\n# \nloaded_lora_weights = torch.load("/kaggle/input/lora-weights-last/lora_weights_last (1).pth", map_location="cpu")\npipe.unet.load_state_dict(loaded_lora_weights, strict=False)\n\n#  start_epoch\nSTART_EPOCH = 5\n\n# train_loop \ndef train_loop_resume_from_5():\n    global_step = 0\n    os.makedirs(OUTPUT_DIR, exist_ok=True)\n    pipe.unet.train()\n    vae = pipe.vae.to(accelerator.device, dtype=torch.float16)\n    loss_history = []\n\n    def clean_old_files(pattern, keep=2):\n        files = sorted(glob.glob(os.path.join(OUTPUT_DIR, pattern)))\n        if len(files) > keep:\n            for f in files[:-keep]:\n                os.remove(f)\n\n    reference_prompt = "A young man with short hair and a soft expression, looking directly at the camera. He is not wearing glasses."\n\n    for epoch in range(START_EPOCH, NUM_EPOCHS):\n        epoch_losses = []\n        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)

In [10]:
"""
train_loop_resume_from_5()
"""

'\ntrain_loop_resume_from_5()\n'

In [11]:
checkpoint_path = "/kaggle/input/lora-weights-last/checkpoint_last_25.pth"
START_EPOCH = 0
NUM_EPOCHS = 30
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location="cpu")

    pipe.unet.load_state_dict(checkpoint["lora_weights"], strict=False)
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    lr_scheduler.load_state_dict(checkpoint["lr_scheduler_state"])

    START_EPOCH = checkpoint.get("epoch", 0) + 1
    print(f" Loaded checkpoint from epoch {START_EPOCH - 1}")
else:
    print("No checkpoint found, starting from scratch.")


 Loaded checkpoint from epoch 24


In [12]:

def train_loop_resume_from_25():
    global_step = 0
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    pipe.unet.train()
    vae = pipe.vae.to(accelerator.device, dtype=torch.float16)
    loss_history = []

    def clean_old_files(pattern, keep=2):
        files = sorted(glob.glob(os.path.join(OUTPUT_DIR, pattern)))
        if len(files) > keep:
            for f in files[:-keep]:
                os.remove(f)

    reference_prompt = "A young man with short hair and a soft expression, looking directly at the camera. He is not wearing glasses."

    for epoch in range(START_EPOCH, NUM_EPOCHS):
        epoch_losses = []
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process)

        for batch in progress_bar:
            with accelerator.accumulate(pipe.unet):
                images = batch["image"].to(accelerator.device, dtype=torch.float16)

                with torch.no_grad():
                    latents = vae.encode(images).latent_dist.sample() * 0.18215

                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.size(0),), device=latents.device)
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                encoded = tokenizer(batch["prompt"], padding="max_length", truncation=True,
                                    max_length=tokenizer.model_max_length, return_tensors="pt")
                input_ids = encoded.input_ids.to(accelerator.device)
                attention_mask = encoded.attention_mask.to(accelerator.device)

                encoder_hidden_states = pipe.text_encoder(input_ids=input_ids, attention_mask=attention_mask)[0]

                with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                    noise_pred = pipe.unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states)[0]
                    loss = torch.nn.functional.mse_loss(noise_pred, noise)

                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(pipe.unet.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad(set_to_none=True)

                epoch_losses.append(loss.item())
                progress_bar.set_postfix(loss=loss.item(), lr=lr_scheduler.get_last_lr()[0])
                accelerator.log({"loss": loss.item(), "lr": lr_scheduler.get_last_lr()[0]}, step=global_step)

                global_step += 1

        avg_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Epoch {epoch} - Average Loss: {avg_loss:.4f}")

        if accelerator.is_main_process:
            loss_history.append({"epoch": epoch, "average_loss": avg_loss})
            with open(os.path.join(OUTPUT_DIR, "loss_log.json"), "w") as f:
                json.dump(loss_history, f, indent=2)

            lora_path = os.path.join(OUTPUT_DIR, f"lora_weights_epoch{epoch}.pth")
            torch.save({k: v for k, v in pipe.unet.state_dict().items() if "lora_" in k or "lora." in k}, lora_path)
            clean_old_files("lora_weights_epoch*.pth")

            with torch.autocast("cuda"):
                generated_image = pipe(reference_prompt, num_inference_steps=30, guidance_scale=7.5).images[0]
                gen_path = os.path.join(OUTPUT_DIR, f"preview_epoch{epoch}.png")
                generated_image.save(gen_path)

    if accelerator.is_main_process:
        final_ckpt = os.path.join(OUTPUT_DIR, f"checkpoint_last_{NUM_EPOCHS}.pth")
        torch.save({
            "epoch": NUM_EPOCHS - 1,
            "lora_weights": {
                k: v for k, v in pipe.unet.state_dict().items()
                if "lora_" in k or "lora." in k
            },
            "optimizer_state": optimizer.state_dict(),
            "lr_scheduler_state": lr_scheduler.state_dict()
        }, final_ckpt)
        print("Training complete and checkpoint saved.")


In [13]:
train_loop_resume_from_25()

Epoch 25:   0%|          | 0/450 [00:00<?, ?it/s]

Epoch 25 - Average Loss: 0.1414


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 26:   0%|          | 0/450 [00:00<?, ?it/s]

Epoch 26 - Average Loss: 0.1368


...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 29:   0%|          | 0/450 [00:00<?, ?it/s]

Epoch 29 - Average Loss: 0.1375


  0%|          | 0/30 [00:00<?, ?it/s]

Training complete and checkpoint saved.


In [14]:
""" # Test difference
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model
import torch
from PIL import Image
from torchvision.utils import save_image
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
).to(device)

prompt = "A close-up portrait of a young woman with curly black hair, individual hair strand detail visible, subtle freckles across her nose and cheeks, wearing a navy headscarf, soft studio lighting, neutral background"
generator = torch.manual_seed(42)

image_base = pipe(prompt, num_inference_steps=50, guidance_scale=9, generator=generator).images[0]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "to_q", "to_k", "to_v", "to_out.0",
        "proj_in", "proj_out",
        "conv1", "conv2", "conv_shortcut"
    ],
    lora_dropout=0.1,
    bias="none",
)
pipe.unet = get_peft_model(pipe.unet, lora_config)

lora_weights_path ="/kaggle/working/lora_weights_last.pth"
state_dict = torch.load(lora_weights_path, map_location=device)
pipe.unet.load_state_dict(state_dict, strict=False)

generator = torch.manual_seed(42)
image_lora = pipe(prompt, num_inference_steps=50, guidance_scale=9, generator=generator).images[0]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].imshow(image_base)
axs[0].set_title("Without LoRA")
axs[0].axis("off")
axs[1].imshow(image_lora)
axs[1].set_title("With LoRA")
axs[1].axis("off")
plt.tight_layout()
plt.show()
"""

' # Test difference\nfrom diffusers import StableDiffusionPipeline\nfrom peft import LoraConfig, get_peft_model\nimport torch\nfrom PIL import Image\nfrom torchvision.utils import save_image\nimport matplotlib.pyplot as plt\n\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\n\npipe = StableDiffusionPipeline.from_pretrained(\n    "runwayml/stable-diffusion-v1-5",\n    torch_dtype=torch.float16,\n    safety_checker=None,\n    requires_safety_checker=False\n).to(device)\n\nprompt = "A close-up portrait of a young woman with curly black hair, individual hair strand detail visible, subtle freckles across her nose and cheeks, wearing a navy headscarf, soft studio lighting, neutral background"\ngenerator = torch.manual_seed(42)\n\nimage_base = pipe(prompt, num_inference_steps=50, guidance_scale=9, generator=generator).images[0]\n\nlora_config = LoraConfig(\n    r=16,\n    lora_alpha=32,\n    target_modules=[\n        "to_q", "to_k", "to_v", "to_out.0",\n        "proj_in", "proj_out",\

PROMPTS:                                                                                              
“A close-up portrait of a young woman with curly black hair, individual hair strand detail visible, subtle freckles across her nose and cheeks, wearing a navy headscarf, soft studio lighting, neutral background”

“An elderly man with deep facial wrinkles and silver stubble, clear blue eyes with crisp eyelashes, wearing rectangular glasses, high-resolution skin texture, clean background”

“A portrait of a toddler girl with braided blonde pigtails, translucent baby skin showing fine pores, detailed eyelashes, wearing a pastel pink dress, natural window light”

“A middle-aged Asian man with salt-and-pepper beard stubble, defined pores and skin texture, wearing a gray turtleneck, sharp focus on eyes”

“A headshot of a woman with perfectly applied red lipstick and green eyeshadow, individual eyelash and eyebrow hairs, glossy lips reflecting light, dramatic dark background”

“A close-up of a young man with buzz-cut hair and pierced ears, silver hoop earrings, vivid iris colors, sharp catchlights in the eyes, clean white backdrop”

“A teenage boy with acne-prone skin showing subtle blemishes, short hair with blonde highlights, wearing a black hoodie, moody side lighting”

“An elderly woman with braided gray hair, delicate crow’s-feet wrinkles around her eyes, vintage round glasses, floral scarf, high-detail portrait”

“A dynamic portrait of a woman with wind-swept hair showing individual strands, wearing a flowing red scarf, autumn leaves softly blurred in background”

“A group portrait of three siblings—one with tight curls, one with straight hair, one with wavy strands—distinct facial features, soft window-lit room, shallow depth of field”